**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
from scripts.get_pocket import *
from scripts.dogsitescorer import *
import numpy as np
import os

software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_main/2o1x_A_apo_protoss.pdb'
ref_file = '/home/tony/CADD22/wocondock_main/2o1x_A_lig_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_main/500_of_FCHGroup_LeadLike.sdf'
docking_programs = ['SMINA','GNINA','PLANTS']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_test_1k/receptor_protoss_prepared.pdb'
ref_file = '/home/tony/CADD22/wocondock_test_1k/crystal_ligand_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_test_1k/merged_actives_decoys.sdf'
docking_programs = ['SMINA','GNINA','PLANTS']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_performance_ace/receptor_protoss_prepared.pdb'
ref_file = '/home/tony/CADD22/wocondock_performance_ace/crystal_ligand_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_performance_ace/merged_actives_decoys.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', 'USRCAT', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[09:20:42] Initializing Normalizer


The working directory has been set to: /home/tony/CADD22/wocondock_performance_ace
The folder: /home/tony/CADD22/wocondock_performance_ace/temp already exists


In [ ]:
pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
cleaned_pkasolver_df = prepare_library(docking_library, id_column, software, 'pkasolver')

In [ ]:
all_poses = docking(protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
docking_splitted(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
all_poses = fetch_poses_splitted(w_dir, n_poses, '/home/tony/CADD22/wocondock_performance_aldr/temp/split_final_library')

In [ ]:
all_poses = pd.concat([PandasTools.LoadSDF('/home/tony/CADD22/wocondock_performance_ace/temp/gnina/docked.sdf', idName='Pose ID', molColName='Molecule',includeFingerprints=False, embedProps=False, removeHs=False, strictParsing=True), 
                       PandasTools.LoadSDF('/home/tony/CADD22/wocondock_performance_ace/temp/smina/docked.sdf', idName='Pose ID', molColName='Molecule',includeFingerprints=False, embedProps=False, removeHs=False, strictParsing=True), 
                       PandasTools.LoadSDF('/home/tony/CADD22/wocondock_performance_ace/temp/plants/results/docked_poses.sdf', idName='Pose ID', molColName='Molecule',includeFingerprints=False, embedProps=False, removeHs=False, strictParsing=True)])
PandasTools.WriteSDF(all_poses, w_dir+"/temp/allposes.sdf", molColName='Molecule', idName='Pose ID', properties=list(all_poses.columns))

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


In [ ]:
for metric in ['bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']:
    cluster(f'{metric}', 'Kmedoids', w_dir, protein_file, all_poses)

**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [4]:
for metric in ['bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']:
    rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 1)


The folder: /home/tony/CADD22/wocondock_performance_ace/temp/rescoring_bestpose_clustered was created
The folder: /home/tony/CADD22/wocondock_performance_ace/temp/rescoring_bestpose_clustered/gnina_rescoring/ was created
Splitting bestpose_clustered.sdf...
Rescoring with GNINA
Splitting SDF file bestpose_clustered.sdf ...


FileNotFoundError: [Errno 2] No such file or directory: '/home/tony/CADD22/wocondock_performance_ace/temp/rescoring_bestpose_clustered/gnina_rescoring/temp/split_bestpose_clustered'

**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
apply_consensus_methods(w_dir, clustering_metrics)

In [ ]:
calculate_EFs(w_dir, docking_library)